##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Extract structured data using function calling

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/tutorials/extract_structured_data"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on Google AI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemini-api/tutorials/extract_structured_data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/gemini-api/tutorials/extract_structured_data.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In this tutorial you'll work through a structured data extraction example, using the Gemini API to extract the lists of characters, relationships, things, and places from a story.

## Setup

In [1]:
!pip install -U -q google-generativeai

In [2]:
import json
import pathlib
import textwrap

import google.generativeai as genai


from IPython.display import display
from IPython.display import Markdown

from google.api_core import retry

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`


In [3]:
try:
    # Used to securely store your API key
    from google.colab import userdata

    # Or use `os.getenv('API_KEY')` to fetch an environment variable.
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
except ImportError:
    pass


## The example task

For this tutorial you'll extract entities from natural language stories. As an
 example, below is a story written by Gemini.

In [ ]:
new_story = False

if new_story:
  model = genai.GenerativeModel(model_name='models/gemini-1.5-pro-latest')

  response = model.generate_content("""
      Write a long story about a girl with magic backpack, her family, and at
      least one other charater. Make sure everyone has names. Don't forget to
      describe the contents of the backpack, and where everyone and everything
      starts and ends up.""", request_options={'retry': retry.Retry()})
  story = response.text
  print(response.candidates[0].citation_metadata)
else:
  story = """In the quaint town of Willow Creek, nestled amidst rolling hills and whispering willows, resided a young girl named Anya. As she stepped out of the creaky wooden door of her modest cottage, her heart skipped a beat with excitement and anticipation. Today was her first day of school, and she couldn't wait to show off her prized possession - a magical backpack.\n\nHanded down to her from her grandmother, the backpack was no ordinary satchel. Its soft, emerald-green fabric shimmered with an ethereal glow, and its leather straps held secrets that only Anya knew. Within its capacious interior lay an enchanted world, filled with wonders that would ignite her imagination and change her life forever.\n\nAnya's parents, kind-hearted Elise and wise-bearded Edward, bid her farewell with warm embraces. "Remember, my dear," whispered her mother, "use your magic wisely and for good." Her father added, "Always seek knowledge, and let the backpack be your trusted companion."\n\nWith a skip in her step, Anya set off towards the town's only schoolhouse. On her way, she passed her best friend, Samuel, a curious and adventurous boy with a mischievous grin. "Hey, Anya," he called out. "Can I see your backpack?"\n\nAnya hesitated for a moment before unzipping the flap and revealing its contents. Samuel's eyes widened in amazement as he peered inside. There, nestled amidst pencils and notebooks, were a shimmering sword, a book of ancient spells, a tiny compass that always pointed north, and a magical key that could open any lock.\n\nTogether, they marveled at the backpack's wonders, promising to keep its secrets safe. As they approached the schoolhouse, Anya noticed a group of older children huddled together, their faces etched with fear. Curiosity getting the better of her, she cautiously approached.\n\n"What's wrong?" she asked.\n\nA tall, lanky boy stepped forward. "There's a monster in the forest," he stammered. "It's been terrorizing the town, attacking animals and even people."\n\nAnya's heart sank. The town of Willow Creek was small and peaceful, and the thought of a monster brought a shiver down her spine. She knew she had to do something to protect her family and friends.\n\nWithout a moment's hesitation, Anya opened her backpack and retrieved the shimmering sword. With a determined gleam in her eye, she turned to her terrified peers. "Don't worry," she said, her voice steady. "I'll take care of it."\n\nWith Samuel close behind her, Anya ventured into the shadowy depths of the forest. The trees seemed to whisper secrets as she passed, and the undergrowth rustled with unseen creatures. As they walked deeper into the forest, the air grew heavy and the ground beneath their feet trembled.\n\nSuddenly, they came to a clearing, and there before their eyes was the monster - a massive beast with sharp teeth, glowing red eyes, and claws that could crush a human with ease. The creature roared, a thunderous sound that shook the forest to its core.\n\nFear surged through Anya, but she refused to let it consume her. She drew the sword from its sheath and charged towards the monster. The blade shimmered in the sunlight, and as it struck the beast's hide, a blinding light erupted, enveloping everything in its radiance.\n\nWhen the light faded, the monster was gone, and in its place was a pile of shattered crystals. Anya had defeated the creature with the magic of her backpack, proving that even the smallest of objects could hold the greatest of powers.\n\nAs she and Samuel returned to the town, they were greeted as heroes. The people of Willow Creek rejoiced, and the legend of Anya, the girl with the magic backpack, was passed down through generations. And so, Anya continued her adventures, using the backpack's wonders to make the world a better place, one magical step at a time."""

In [ ]:
to_markdown(story)

> In the quaint town of Willow Creek, nestled amidst rolling hills and whispering willows, resided a young girl named Anya. As she stepped out of the creaky wooden door of her modest cottage, her heart skipped a beat with excitement and anticipation. Today was her first day of school, and she couldn't wait to show off her prized possession - a magical backpack.
> 
> Handed down to her from her grandmother, the backpack was no ordinary satchel. Its soft, emerald-green fabric shimmered with an ethereal glow, and its leather straps held secrets that only Anya knew. Within its capacious interior lay an enchanted world, filled with wonders that would ignite her imagination and change her life forever.
> 
> Anya's parents, kind-hearted Elise and wise-bearded Edward, bid her farewell with warm embraces. "Remember, my dear," whispered her mother, "use your magic wisely and for good." Her father added, "Always seek knowledge, and let the backpack be your trusted companion."
> 
> With a skip in her step, Anya set off towards the town's only schoolhouse. On her way, she passed her best friend, Samuel, a curious and adventurous boy with a mischievous grin. "Hey, Anya," he called out. "Can I see your backpack?"
> 
> Anya hesitated for a moment before unzipping the flap and revealing its contents. Samuel's eyes widened in amazement as he peered inside. There, nestled amidst pencils and notebooks, were a shimmering sword, a book of ancient spells, a tiny compass that always pointed north, and a magical key that could open any lock.
> 
> Together, they marveled at the backpack's wonders, promising to keep its secrets safe. As they approached the schoolhouse, Anya noticed a group of older children huddled together, their faces etched with fear. Curiosity getting the better of her, she cautiously approached.
> 
> "What's wrong?" she asked.
> 
> A tall, lanky boy stepped forward. "There's a monster in the forest," he stammered. "It's been terrorizing the town, attacking animals and even people."
> 
> Anya's heart sank. The town of Willow Creek was small and peaceful, and the thought of a monster brought a shiver down her spine. She knew she had to do something to protect her family and friends.
> 
> Without a moment's hesitation, Anya opened her backpack and retrieved the shimmering sword. With a determined gleam in her eye, she turned to her terrified peers. "Don't worry," she said, her voice steady. "I'll take care of it."
> 
> With Samuel close behind her, Anya ventured into the shadowy depths of the forest. The trees seemed to whisper secrets as she passed, and the undergrowth rustled with unseen creatures. As they walked deeper into the forest, the air grew heavy and the ground beneath their feet trembled.
> 
> Suddenly, they came to a clearing, and there before their eyes was the monster - a massive beast with sharp teeth, glowing red eyes, and claws that could crush a human with ease. The creature roared, a thunderous sound that shook the forest to its core.
> 
> Fear surged through Anya, but she refused to let it consume her. She drew the sword from its sheath and charged towards the monster. The blade shimmered in the sunlight, and as it struck the beast's hide, a blinding light erupted, enveloping everything in its radiance.
> 
> When the light faded, the monster was gone, and in its place was a pile of shattered crystals. Anya had defeated the creature with the magic of her backpack, proving that even the smallest of objects could hold the greatest of powers.
> 
> As she and Samuel returned to the town, they were greeted as heroes. The people of Willow Creek rejoiced, and the legend of Anya, the girl with the magic backpack, was passed down through generations. And so, Anya continued her adventures, using the backpack's wonders to make the world a better place, one magical step at a time.

## Using Natural language

Large language models are a powerfuls multitask tools. Often you can just ask Gemini for what you want, and it will do okay.  

The Gemini API doesn't have a JSON mode, so there are a few things to watch for when generating data structures this way:

- Sometimes parsing fails.
- The schema can't be strictly enforced.

You'll solve those problems in the next section. First, try a simple natural language prompt with the schema written out as text. This has not been optimized:

In [4]:
import requests
from bs4 import BeautifulSoup

# Send a GET request to the specified URL
response = requests.get("https://www.orcanetwork.org/recent-sightings")

html_content = response.content

# Use Beautiful Soup to parse the HTML
soup = BeautifulSoup(html_content, "html.parser")



In [5]:
!pip install markdownify



In [6]:
element = soup.find_all('div',class_='sqs-html-content')
e = element[3]

all_content = e.contents  # or use list(element.children)

res = ""
# Print results
for content in all_content:
    res += str(content.get_text())
    res += "\n"

    # print(content)


In [7]:
from markdownify import markdownify as md
mark_d = md("".join(str(item) for item in all_content))


In [8]:
res_split = mark_d.split('**December')

In [9]:
print(len(res_split))

31


In [10]:
model = genai.GenerativeModel(
    model_name='models/gemini-1.5-pro-latest')

json_res = ""

for i in range(len(res_split)):
  response = model.generate_content(
    textwrap.dedent("""\
      Please return JSON describing the the whale sightings reported by users across the world using the following schema:

    {"Date": "The date when the orca or whale sighting occurred, formatted as MM/DD/YYYY.",
    "Time": "The time of the sighting, typically in 12-hour format with AM/PM indication.",
    "Species": "The species or specific ecotype of whale or orca observed (e.g., Southern Residents, Bigg's).",
    "Location": "The reported location of the sighting, often including landmarks or geographic areas.",
    "Latitude": "The latitude coordinate of the sighting location, measured in decimal degrees.",
    "Longitude": "The longitude coordinate of the sighting location, measured in decimal degrees.",
    "Number of Animals": "The number of whales or orcas observed during the sighting, if reported.",
    "Direction of Travel": "The general direction in which the animals were moving (e.g., North, South, Northwest).",
    "Behaviors Observed": "Observations about the behavior of the animals, such as traveling, milling, hunting, or spyhopping.",
    "Males?": "Indicates whether any males were identified among the group, if known.",
    "Notes": "Additional details about the sighting, such as observer information, unusual features, or actions.",
    "Links": "References or URLs related to the sighting, if available.",
    "Raw_Text: "The unprocessed text from which the JSON was derived."
    }

    For the latitude and longitude column use the location to infer these values

      All fields are required.

      Important: Only return a single piece of valid JSON text.

      Here are the sightings reported in markdown:

      """) + res_split[i],
    generation_config={'response_mime_type':'application/json'}
  )
  json_res += response.text

print(json_res)

KeyboardInterrupt: 

In [27]:
import asyncio
import time
from tqdm.auto import tqdm
from typing import List, Dict, Union
from functools import partial

async def _async_prompt(
    model:genai.GenerativeModel,
    prompt:str,
    temperature:float=0.1,
    max_output_tokens:int=8192,
    top_p:float=1.0,
    top_k:int=40
):
    """Perform asynchronous call to the Gemini API

    Parameters:
        prompt (str): Prompt
        return_json (bool): Whether expecting the output to be in json format or not
        temperature (float): Temperature parameter
        max_output_tokens (int): Maximum tokens to generate (Gemini models have a maximum of 8192)
        top_p (float): top_p parameter
        top_k (int): top_k
    Returns:
        Union[Dict|str]: The output from Gemini (either a dictionary or string)
    """
    try:
        response = await model.generate_content_async(
            prompt,
            generation_config={
                "max_output_tokens": max_output_tokens,
                "temperature": temperature,
                "top_p": top_p,
                "top_k": top_k,
                'response_mime_type':'application/json'
            },
            stream=False
        )

        return response.text
    except ValueError:
        return ""


async def parallel_prompt(
    model:genai.GenerativeModel,
    prompts:List[str],
    temperature:float=0.1,
    max_output_tokens:int=8192,
    top_p:float=1.0,
    top_k:int=40,
    calls_per_minute:int=15,
    verbose:bool=False
) :
    """Call the Gemini API in parallel, while respecting the API's RPM limits

    Parameters:
        prompts (List[str]): List of prompts (strings)
        return_json (bool): Whether expecting the output to be in json format or not
        temperature (float): Temperature parameter
        max_output_tokens (int): Maximum tokens to generate (Gemini models have a maximum of 8192)
        top_p (float): top_p parameter
        top_k (int): top_k
        calls_per_minute (int): The number of calls per minute (or Rate Per Minute)
        verbose (bool): Flag for verbose output
    Returns:
        Union[List[Dict]|List[str]]: The output as a List of either dictionaries or strings
    """
    num_prompts = len(prompts)

    # Currently have to subtract 1 from calls_per_minute to avoid 429 error
    batch_size = min(calls_per_minute, num_prompts)

    # Wait time in seconds
    wait_time = 60

    # Using partial to "fill in" the parameters to _async_prompt
    async_prompt_partial = partial(
        _async_prompt,
        model=model,
        temperature=temperature,
        max_output_tokens=max_output_tokens,
        top_p=top_p,
        top_k=top_k
    )

    prompt_results = []

    # Batches are made in relation to the RPM limit
    # If the Gemini API returns within 1 minute, will sleep until the full minute is over to avoid 429 error
    for i in tqdm(range(0, num_prompts, batch_size), disable=not verbose):

        start_time = time.time()

        batch = prompts[i:i+batch_size]

        # Using asyncio.gather to perform multiple calls simultaneously
        responses = await asyncio.gather(*[async_prompt_partial(prompt=prompt) for prompt in batch])

        prompt_results += responses

        if len(prompt_results) < len(prompts):
            elapsed_time = time.time() - start_time

            # Add 1 second to wait_time to avoid 429 errors
            sleep_time = max(0, wait_time+1 - elapsed_time)

            if sleep_time > 0:
                if verbose:
                    print(f"Waiting for {sleep_time} seconds to respect API quota")

                await asyncio.sleep(sleep_time)

    return prompt_results

In [14]:
response = """\
      Please return JSON describing the the whale sightings reported by users across the world using the following schema:

    {"Date": "The date when the orca or whale sighting occurred, formatted as MM/DD/YYYY.",
    "Time": "The time of the sighting, typically in 12-hour format with AM/PM indication.",
    "Species": "The species or specific ecotype of whale or orca observed (e.g., Southern Residents, Bigg's).",
    "Location": "The reported location of the sighting, often including landmarks or geographic areas.",
    "Latitude": "The latitude coordinate of the sighting location, measured in decimal degrees.",
    "Longitude": "The longitude coordinate of the sighting location, measured in decimal degrees.",
    "Number of Animals": "The number of whales or orcas observed during the sighting, if reported.",
    "Direction of Travel": "The general direction in which the animals were moving (e.g., North, South, Northwest).",
    "Behaviors Observed": "Observations about the behavior of the animals, such as traveling, milling, hunting, or spyhopping.",
    "Males?": "Indicates whether any males were identified among the group, if known.",
    "Notes": "Additional details about the sighting, such as observer information, unusual features, or actions.",
    "Links": "References or URLs related to the sighting, if available.",
    "Raw_Text: "The unprocessed text from which the JSON was derived."
    }

    For the latitude and longitude column use the location to infer these values

      All fields are required.

      Important: Only return a single piece of valid JSON text.

      Here are the sightings reported in markdown:"""

In [23]:
# creating prompts
multiple_prompts = []

for i in range(len(res_split)):

  if (len(res_split[i].strip()) == 0):
    continue

  prompt = response
  prompt += "\n"
  prompt = prompt + res_split[i]

  multiple_prompts.append(prompt)

In [ ]:
start_time = time.time()

# Running 15 prompts (within the 15 RPM limit) simultaneously - should take a few seconds
example_output = await parallel_prompt(model, multiple_prompts)

end_time = time.time() - start_time

print(f"Processing 30 prompts took {round(end_time, 2)} seconds")

That returned a json string. Try parsing it:

In [ ]:
print(json.dumps(json.loads(response.text), indent=4))

{
    "people": [
        {
            "name": "Anya",
            "description": "A young girl with a magical backpack",
            "start_place_name": "Willow Creek",
            "end_place_name": "Willow Creek"
        },
        {
            "name": "Elise",
            "description": "Anya's kind-hearted mother",
            "start_place_name": "Willow Creek",
            "end_place_name": "Willow Creek"
        },
        {
            "name": "Edward",
            "description": "Anya's wise-bearded father",
            "start_place_name": "Willow Creek",
            "end_place_name": "Willow Creek"
        },
        {
            "name": "Samuel",
            "description": "Anya's curious and adventurous best friend",
            "start_place_name": "Willow Creek",
            "end_place_name": "Willow Creek"
        }
    ],
    "places": [
        {
            "name": "Willow Creek",
            "description": "A quaint town nestled amidst rolling hills and whispering w

That's relatively simple and often works, but you can potentially make this more strict/robust by defining the schema using the API's function calling feature.

## Use function calling

If you haven't gone through the [Function calling basics](https://ai.google.dev/tutorials/function_calling_python_quickstart) tutorial yet, make sure you do that first.

With function calling your function and its parameters are described to the API as a `genai.protos.FunctionDeclaration`. In basic cases the SDK can build the `FunctionDeclaration` from the function and its annotations. The SDK doesn't currently handle the description of nested `OBJECT` (`dict`) parameters. So you'll need to define them explicitly, for now.

### Define the schema

Start by defining `person` as an object with string fields `name`, `description`, `start_place_name`, `end_place_name`.

In [ ]:
from typing_extensions import TypedDict

In [ ]:
class Person(TypedDict):
    name: str
    description: str
    start_place_name: str
    end_place_name: str

Then do the same for each of the entities you're trying to extract:

In [ ]:
class Place(TypedDict):
    name:str
    description:str

In [ ]:
class Thing(TypedDict):
    name:str
    description:str


In [ ]:
class Relationship(TypedDict):
    person_1_name: str
    person_2_name: str
    relationship: str

Now build the `FunctionDeclaration`:

In [ ]:
def add_to_database(
    people: list[Person],
    places: list[Place],
    things: list[Thing],
    relationships: list[Relationship]
):
    pass

### Call the API

Like you saw in [Function calling basics](https://ai.google.dev/tutorials/function_calling_python_quickstart) now you can pass this `FunctionDeclaration` to the `tools` argument of the `genai.GenerativeModel` constructor (the constructor would also accept an equivalent JSON representation of the function declaration):

In [ ]:
model = genai.GenerativeModel(
    model_name='models/gemini-1.5-pro-latest',
    tools = [add_to_database])

Each time you call the API the SDK will send the tools along with your prompt, and the model should call that function you defined:

In [ ]:
result = model.generate_content(f"""
{story}

Please add the people, places, things, and relationships from this story to the database:
""",
# Force a function call
# tool_config={'function_calling_config':'ANY'}
)

Now there is no text to parse. The result _is_ a datastructure.

In [ ]:
'text' in result.candidates[0].content.parts[0]

False

In [ ]:
'function_call' in result.candidates[0].content.parts[0]

True

In [ ]:
fc = result.candidates[0].content.parts[0].function_call
print(type(fc))

<class 'google.ai.generativelanguage_v1beta.types.content.FunctionCall'>


The `genai.protos.FunctionCall` class is based on Google Protocol Buffers, convert it to a more familiar JSON compatible object:

In [ ]:
print(json.dumps(type(fc).to_dict(fc), indent=4))

{
    "name": "add_to_database",
    "args": {
        "people": [
            {
                "description": "A young girl with a magical backpack.",
                "name": "Anya"
            },
            {
                "description": "Anya\\'s kind-hearted mother.",
                "name": "Elise"
            },
            {
                "description": "Anya\\'s wise-bearded father.",
                "name": "Edward"
            },
            {
                "description": "Anya\\'s curious and adventurous best friend.",
                "name": "Samuel"
            }
        ],
        "things": [
            {
                "description": "A backpack with magical contents, including a shimmering sword, a book of ancient spells, a tiny compass, and a magical key.",
                "name": "Magical backpack"
            },
            {
                "description": "A sword that emits a blinding light when it strikes.",
                "name": "Shimmering sword"
   

## Conclusion

While the API can handle structured data extraction problems with pure text input and text output, using function calling is likely more reliable since it lets you define a strict schema, and eliminates a potentially error-prone parsing step.